In [21]:
import os
import json
import cv2
import string
import random
import albumentations as A
import copy
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [63]:
template_directory = 'templates_img'

text_locations = json.load(open('text_locations.json', 'r'))

image_dir =  "data/"

backgrounds_dir = 'image_backgrounds'

In [23]:
def generate_random_field(length = 0):
    length = length if length else random.randint(2, 20)
    field = ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation, k=length))
    return field

generate_random_field()

'i-Sg>+"{d'

In [51]:
transform = A.Compose([
        # A.RandomRotate90(),
        # A.Flip(),
        # A.Transpose(),
        A.ImageCompression(quality_lower=10, p=0.1),
        A.OneOf([
            # A.IAAAdditiveGaussianNoise(),
            A.GaussNoise(p=0.8),
            A.ISONoise(p=0.2),
            A.MultiplicativeNoise(p=.05)
        ], p=0.1),
        A.OneOf([
            A.MotionBlur(p=.2),
            A.MedianBlur(blur_limit=3, p=0.1),
            A.Blur(blur_limit=3, p=0.1),
        ], p=0.05),
        A.ShiftScaleRotate(shift_limit=0.15, scale_limit=0.2, rotate_limit=30, p=0.5),
        A.OneOf([
            A.OpticalDistortion(p=0.5),
            A.GridDistortion(p=.5),
            A.PiecewiseAffine(p=0.5),
        ], p=0.5),
        A.OneOf([
            A.CLAHE(clip_limit=2),
            A.Sharpen(),
            A.Emboss(),
            A.RandomBrightnessContrast(),            
        ], p=0.05),
        A.OneOf([
            A.RandomFog(),
            A.RandomRain(),
            A.RandomSnow(),
            A.RandomSunFlare(),            
        ], p=0.1),
        A.HueSaturationValue(p=0.01),
        # A.Rotate(66,p=0.3)
        # A.ToGray(always_apply=True)
    ])
# random.seed(42) 

In [25]:
fonts = [
    cv2.FONT_HERSHEY_SIMPLEX,
    cv2.FONT_HERSHEY_COMPLEX,
    cv2.FONT_HERSHEY_PLAIN,
    cv2.FONT_HERSHEY_DUPLEX,
    cv2.FONT_HERSHEY_TRIPLEX,
    cv2.FONT_HERSHEY_COMPLEX_SMALL,
    cv2.FONT_HERSHEY_SCRIPT_COMPLEX,
    cv2.FONT_HERSHEY_SCRIPT_COMPLEX,
    cv2.FONT_ITALIC]

In [12]:
def generate_target_dictionary():
    with open('data_dictionary.json') as data_dict:
        categories = json.load(data_dict)['target_data']
    return categories
categories = generate_target_dictionary()  

backgrounds = []
for filename in os.listdir(backgrounds_dir):
    img = cv2.imread(backgrounds_dir+ '/'+filename, 1)
    backgrounds.append(img)

test_gray = np.memmap('data_full/test_gray_data.npy', mode='w+', shape = (50000,275,210))
test_binary = np.memmap('data_full/test_binary_data.npy', mode='w+', shape = (50000,275,210))
test_target = np.memmap('data/test_target.npy', mode='w+', shape = (50000))

index = 0
for filename in text_locations:
    if text_locations[filename] != {}:
        image = cv2.imread(template_directory+ '/'+filename, 1)
        image = cv2.resize(image, (850, 1100)) 

        doc_info = text_locations[filename]
        for i in range(10000):
            background_img = backgrounds[random.randint(0, len(backgrounds))-1]
            img = image
            for loc in doc_info:
                font = random.choice(fonts)
                cv2.putText(img, generate_random_field(),
                            (int(loc['x']),int(loc['y'])), font,
                            0.75, (255, 0, 0), 2)
            # resize(210,275)
            x_size = random.randint(-200,400)
            y_size = random.randint(-200,400)
            x_size = x_size if x_size > 0 else 0
            y_size = y_size if y_size > 0 else 0
            x_offset = int(x_size/2)
            y_offset = int(y_size/2)
            background_img = cv2.resize(background_img, (850+x_size, 1100+y_size)) 

            background_img[y_offset:y_offset+img.shape[0], x_offset:x_offset+img.shape[1]] = img
            transformed = transform(image=cv2.resize(background_img, (210,275)))
            img = transformed['image']
            
            img = cv2.cvtColor(transformed['image'], cv2.COLOR_BGR2GRAY)
            test_gray[index] = img
            # print(img.shape)
            img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
            test_binary[index] = img
            test_target[index] = categories[filename[:4]]

            index+=1
            # print(img.shape)
            # cv2.imshow('image', img)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            print(index,end='\r')


In [20]:
out_binary = np.memmap('data_full/test_binary_data.npy',mode='r', shape = (50000,275,210))
# for x in out_binary[:5]:
    # img = cv2.imread(x)

    # cv2.imshow('image', x, cmap='gray')
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
# plt.imshow(out_binary[5], cmap='gray')
for x in range(15):

    cv2.imshow('image', out_binary[np.random.randint(0,out_binary.shape[0])])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [65]:
backgrounds = []
for filename in os.listdir(backgrounds_dir):
    img = cv2.imread(backgrounds_dir+ '/'+filename, 1)
    backgrounds.append(img)

for filename in text_locations:
    if text_locations[filename] != {}:
        image = cv2.imread(template_directory+ '/'+filename, 1)
        image = cv2.resize(image, (850, 1100)) 

        doc_info = text_locations[filename]
        for i in range(3):
            background_img = backgrounds[random.randint(0, len(backgrounds))-1]
            img = image
            for loc in doc_info:
                font = random.choice(fonts)
                cv2.putText(img, generate_random_field(),
                            (int(loc['x']),int(loc['y'])), font,
                            1, (0, 0, 0), 1)
            # resize(210,275)
            x_size = random.randint(-150,400)
            y_size = random.randint(-150,400)
            x_size = x_size if x_size > 150 else 0
            y_size = y_size if y_size > 150 else 0
            x_offset = int(x_size/1.5)
            y_offset = int(y_size/1.5)
            background_img = cv2.resize(background_img, (850+abs(x_size), 1100+abs(y_size))) 

            background_img[y_offset:y_offset+img.shape[0], x_offset:x_offset+img.shape[1]] = img
            transformed = transform(image=cv2.resize(background_img, (210,275)))
            img = transformed['image']

            # img = cv2.resize(img, (100,100))
            img = cv2.resize(img, (150,150))

            # img = cv2.resize(img, (850+abs(x_size), 1100+abs(y_size)))
            img = cv2.resize(img, (210,275))
            
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)

            cv2.imshow('image', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()